In [1]:
import ibm_watson_studio_lib
wslib = ibm_watson_studio_lib.access_project_or_space()

In [2]:
#接続オブジェクトのリスト
assets=wslib.list_connections()
wslib.show(assets)

# 0
{'name': '200322icosbucket',
 'description': '',
 'asset_id': 'aa0c18a3-2833-42ba-aa53-010d5aa2d055',
 'asset_type': 'connection',
 'tags': []}
# 1
{'name': 'DB2WHOC_HDM',
 'description': '',
 'asset_id': 'b06616da-3701-467c-9568-0a56c933155c',
 'asset_type': 'connection',
 'tags': []}


In [3]:
#接続オブジェクトの情報を取得
connection_config=wslib.get_connection('DB2WHOC_HDM')
#connection_config

In [4]:
#SQLAlchemyで接続
import sqlalchemy
from sqlalchemy import *
import ibm_db_sa
con_params = 'ibm_db_sa://%s:%s@%s:%s/%s' % (
    connection_config['username'], connection_config['password'], connection_config['host'], connection_config['port'], connection_config['database'])
if connection_config['ssl']=='true' :
    con_params = con_params+ ";Security=SSL"
con_sa = sqlalchemy.create_engine(con_params)

In [5]:
# データフレームに検索結果の読み込み
import pandas as pd
# SELECT文の実行
sql = "select 2 as ID, 'suzuki' as NAME, 0.1 as RATE, SYSDATE as TDATE from SYSIBM.SYSDUMMY1" \
   " union all select 1 as ID, 'sato' as NAME, 2.5 as RATE, SYSDATE as TDATE from SYSIBM.SYSDUMMY1"

df = pd.read_sql(sql, con_sa)
#df['name'] = df['name'].astype("string")
df

,id,name,rate,tdate
0,1,sato,2.5,2022-02-01 12:57:56
1,2,suzuki,0.1,2022-02-01 12:57:56


In [6]:
#データタイプの確認
df.dtypes

id                int64
name             object
rate            float64
tdate    datetime64[ns]
dtype: object

In [7]:
#if_exists='replace'が動作しないので削除をしていく
sql = text('DROP TABLE IF EXISTS hkwddummy;')
result = con_sa.execute(sql)

In [8]:
#object型をCLOB型にしないためにVARCHAR(255)で指定。
cols = df.dtypes[df.dtypes=='object'].index
type_mapping = {col : String(255) for col in cols }
#dfのテーブルへの書き込み
df.to_sql(name='hkwddummy', con=con_sa, schema='hkwd', if_exists='replace', dtype=type_mapping)


https://jpndev.com/post/431010
<br>
https://www.ibm.com/docs/ja/db2/11.5?topic=psdf-verifying-python-driver-sqlalchemy-adapter-django-adapter-installation

In [9]:
#作成されたテーブルのスキーマ情報の確認
inspector = sqlalchemy.inspect(con_sa)
inspector.get_columns("hkwddummy")

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False},
 {'name': 'id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False},
 {'name': 'name',
  'type': VARCHAR(length=255),
  'nullable': True,
  'default': None,
  'autoincrement': False},
 {'name': 'rate',
  'type': DOUBLE(),
  'nullable': True,
  'default': None,
  'autoincrement': False},
 {'name': 'tdate',
  'type': TIMESTAMP(),
  'nullable': True,
  'default': None,
  'autoincrement': False}]

SQLAlchemyを利用して既存テーブルのメタデータを取得する - 
<br>
https://qiita.com/kurukuruz/items/4d3569edf40582259d1b

In [10]:
con_sa.dispose()

In [11]:
import sys

print(sys.version)

3.8.11 (default, Aug  3 2021, 15:09:35) 
[GCC 7.5.0]


In [12]:
print(pd.__version__)

1.2.4


In [13]:
sqlalchemy.__version__

'1.4.22'

In [14]:
ibm_watson_studio_lib.__version__

'3.0.6'

In [15]:
ibm_db_sa.__version__

'0.3.7'